In [1]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

2023-08-14 17:28:46 GM01X680 metapub.config[477500] WARNING NCBI_API_KEY was not set.


In [2]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [3]:
websites = [
    'ncbi.nlm.nih.gov', 'elsevier.com', 'wiley.com', 'springer.com', 'physiology.org', 'oup.com', 
    'cambridge.org', 'karger.com', 'lww.com', 'nature.com', 'science.org', 'tandfonline.com', 
    'sagepub.com', 'jamanetwork.com', 'neurology.org', 'biorxiv.org', 'royalsocietypublishing.org', 
    'psycnet.apa.org', 'arvojournals.org', 'jstage.jst.go.jp', 'psychiatryonline.org', 'europepmc.org', 
    'mit.edu', 'thejns.org', 'annualreviews.org', 'snmjournals.org', 'aspetjournals.org', 'elibrary.ru', 
    'books.google.de', 'architalbiol.org', 'ahajournals.org', 'liebertpub.com', 'acs.org', 'degruyter.com', 
    'worldscientific.com', 'iospress.com', 'asahq.org', 'thieme-connect.de', 'neurologia.com', 'mpg.de', 
    'opg.optica.org', 'mcgill.ca', 'rbojournal.org', 'taylorfrancis.com', 'ekja.org', 'www.imrpress.com', 
    'theses.fr', 'ieee.org', 'ssrn.com', 'open.bu.edu', 'journals.biologists.com', 'aip.org', 'lib.wfu.edu', 
    'literatumonline.com', 'scholarpedia.org', 'isho.jp', 'mirasmart.com', 'microbiologyresearch.org', 
    'aps.org', 'ujms.net', 'www.ingentaconnect.com', 'symposium.cshlp.org', 'cabdirect.org', 'ajtmh.org'
]

In [4]:
# www.nature.com
def www_nature_com(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Chrome()
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    try:
        elems = driver.find_elements(By.XPATH, "//span[@class='c-bibliographic-information__value')]")
        for elem in elems:
            if "doi.org/" in elem.text:
                doi = elem.text.split("doi.org/")[1]
    except:
        doi = np.nan
    pmid = np.nan
    pmcid = np.nan
    try:
        title = driver.find_element(By.XPATH, "//h1[contains(@class, 'c-article-title')]").text
    except:
        title = np.nan
    abstract = np.nan
    keywords = np.nan
    intro = np.nan
    pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "introduction": intro,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://www.nature.com/articles/387281a0"
# info = www_nature_com(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["introduction"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [5]:
# nature.com
def func_nature_com(url):
    # initialize
    info = {
        "doi": np.nan,
        "pmid": np.nan,
        "pmcid": np.nan,
        "title": np.nan,
        "abstract": np.nan,
        "keywords": np.nan,
        "introduction": np.nan,
        "pdf_link": np.nan
    }

    # set up the webdriver
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome()

    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver.get(url)
            time.sleep(5)
            wait = WebDriverWait(driver, 30)
            wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept All Cookies')]"))).click()
            time.sleep(2)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi, pmid, pmcid
    try:
        doi = driver.find_element(By.XPATH, "//div[contains(@class, 'ej-journal-info')]").text.split("DOI: ")[1]
        doi = doi.strip()
    except:
        doi = np.nan
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.XPATH, "//h1[@class='ejp-article-title']").text
        title = title.strip()
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = ""
        elems = driver.find_element(By.XPATH, "//div[@class='ejp-article-text-abstract']").find_elements(By.TAG_NAME, 'p')
        for elem in elems:
            abstract = abstract + elem.text + " "
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    # try:
    #     keywords = ""
    #     elems = driver.find_element(By.XPATH, "//div[@class='ejp-article-text-abstract']").find_elements(By.TAG_NAME, 'a')
    #     for elem in elems:
    #         keywords = keywords + elem.text + "; "
    #     keywords = keywords.strip()
    # except:
    #     keywords = np.nan
    keywords = np.nan
    
    # # introduction
    # try:
    #     intro = ""
    #     elements = driver.find_elements(By.TAG_NAME, "h2")
    #     for element in elements:
    #         if "Introduction" in element.text:
    #             ele_paren = element.find_element(By.XPATH, "..")
    #             intros = ele_paren.find_elements(By.TAG_NAME, "p")
    #             for intro_ele in intros:
    #                 intro = intro + intro_ele.text + " "
    #             break
    #     intro = intro.strip()
    # except:
    #     intro = np.nan
    intro = np.nan

    # pdf_link
    # try:
    #     pdf_link = driver.find_element(By.XPATH, "//button[contains(text(), 'PDF')]").click()
    #     driver.switch_to_window(driver.window_handles[-1])
    #     pdf_link = driver.current_url
    #     pdf_link = pdf_link.strip()
    # except:
    #     pdf_link = np.nan
    pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "introduction": intro,
        "pdf_link": pdf_link
    }

    return info
# --------------------start of test code--------------------
# # url = 
# # url = 
# # url = 
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["introduction"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [6]:
# science.org

In [7]:
# tandfonline.com

In [8]:
# sagepub.com

In [9]:
# jamanetwork.com

In [10]:
# neurology.org

In [11]:
# biorxiv.org

In [13]:
# extract text from given .pdf file
def pdf2text(pdf_path, page_start, page_end):
    # creating a pdf reader object
    reader = PyPDF2.PdfReader(pdf_path)
    
    # printing number of pages in pdf file
    page_max = len(reader.pages)
    
    # getting a specific page from the pdf file
    text = ""
    
    for i in range(page_start, page_end + 1):
        # print(i)
        page = reader.pages[i]
        text = text + "".join(page.extract_text().splitlines())

    return text
# --------------------start of test code--------------------
# pdf_folder_path = fpath.litera_pdf_folder
# pdf_file_name = "1.pdf"
# pdf_path = os.path.join(pdf_folder_path, pdf_file_name)

# text_folder_path = fpath.litera_text_folder
# text_file_name = pdf_file_name.split(".pdf")[0] + ".txt"
# text_path = os.path.join(text_folder_path, text_file_name)

# start_page = 0
# end_page = 1
# text = pdf2text(pdf_path, start_page, end_page)

# with open(text_path, "w") as f:
#     f.write(text)

# f.close()
# ---------------------end of test code---------------------

In [14]:
# download pdf to specified folder given pdf_url and file name
def download_pdf(pdf_url, pdf_folder_path, file_name):    
    # get the pdf content
    options = webdriver.ChromeOptions()
    options.add_experimental_option('prefs', {
    "download.default_directory": pdf_folder_path, #Change default directory for downloads
    "download.prompt_for_download": False, #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
    })
    driver = webdriver.Chrome(options=options)
    response  = driver.get(pdf_url)
    time.sleep(5)
    # response = requests.get(pdf_url, stream=True, headers=plib.headers)
    
    # download the .pdf file to the pdf_file_path folder
    # write content in pdf file
    # pdf_path = os.path.join(pdf_folder_path, file_name + '.pdf')
    
    # if response.status_code == 200:
    #     with open(pdf_path, 'wb') as pdf_object:
    #         pdf_object.write(response.content)
    #         return True
    # else:
    #     print(f'Failed downloading PDF:' + 'pdf_url')
    #     print(f'HTTP response status code: {response.status_code}')
    #     return False
# --------------------start of test code--------------------
# pdf_url = 'https://www.sciencedirect.com/science/article/pii/S0896627320300052/pdfft?md5=3f0648c6385e6fae3a5a73b053903014&pid=1-s2.0-S0896627320300052-main.pdf'
# pdf_folder_path = fpath.litera_pdf_folder
# file_name = '4'
# download_pdf(pdf_url, pdf_folder_path, file_name)
# ---------------------end of test code---------------------